In [1]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torchvision.transforms as transforms
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch import nn, optim
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from itertools import cycle
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, plot_roc_curve, accuracy_score
import matplotlib.pyplot as plt
import seaborn
import os
import glob
from collections import defaultdict

In [2]:
v1_data_path = f'{os.getcwd()}/../v1/data'

In [3]:
train_files = glob.glob(f'{v1_data_path}/train/*.csv')
test_files = glob.glob(f'{v1_data_path}/test/*.csv')
df_label = pd.read_csv(f'{v1_data_path}/label.csv')

In [4]:
df_label = df_label.sort_values(['RecordID']).reset_index(drop=True)

In [5]:
input_dict = {
    'gender': 0,
    'age_0to2': 1,
    'age_2to5': 2,
    'age_5to12': 3,
    'age_12p': 4,
    'dew_point_avg': 5,
    'humidity_avg': 6,
    'pressure_avg': 7,
    'surveillance': 8,
    'antibiotics': 9,
    'temperature_min': 10,
    'temperature_max': 11,
    'temperature_avg': 12,
    'reducer': 13,
    'fever': 14,
    'isFlu': 15
}

In [6]:
# Train data load
df_train_data = []
train_days = [0] * 100
for idx, train_file in enumerate(train_files):
    df_data = pd.read_csv(train_file)
    count = int(df_data.tail(1)['Time'].iloc[0].split(':')[0]) // 24 + 1
    train_days[count] += 1
    isFlu = df_label[df_label['RecordID'] == (int(os.path.basename(train_file)[:-4])-1)]['flu'].iloc[0]
    df_isFlu = pd.DataFrame([['0:0', 'isFlu', isFlu]], columns=['Time', 'Parameter', 'Value'])
    df_data = pd.concat([df_data, df_isFlu], axis=0,  ignore_index=True)
    df_train_data.append(df_data)

In [7]:
df_train_days = pd.DataFrame(train_days, columns=['DayCount']).reset_index()

In [8]:
df_train_days = df_train_days.drop(index=[0])

In [9]:
df_train_days = df_train_days.rename(columns={'index':'일수'})

In [10]:
df_train_days.to_excel('train_data_daycount.xlsx')

In [11]:
# Test data load
df_test_data = []
test_days = [0] * 40
for test_file in test_files:
    df_data = pd.read_csv(test_file)
    count = int(df_data.tail(1)['Time'].iloc[0].split(':')[0]) // 24 + 1
    print(count,df_data.tail(1)['Time'].iloc[0])
    test_days[count] += 1
    isFlu = df_label[df_label['RecordID'] == (int(os.path.basename(test_file)[:-4])-1)]['flu'].iloc[0]
    df_isFlu = pd.DataFrame([['0:0', 'isFlu', isFlu]], columns=['Time', 'Parameter', 'Value'])
    df_data = pd.concat([df_data, df_isFlu], axis=0,  ignore_index=True)
    df_test_data.append(df_data)

4 86:39
2 47:48
4 79:40
3 52:33
5 114:50
2 25:22
2 28:30
2 36:32
1 23:2
1 22:40
2 41:8
2 32:6
3 67:58
6 121:4
4 73:0
2 47:30
1 12:26
7 151:33
3 66:17
2 43:51
2 30:24
2 36:57
3 58:37
3 50:14
3 59:6
4 92:6
3 50:49
3 70:33
2 26:56
3 57:48
2 27:18
2 31:13
3 63:55
4 88:59
2 46:47
3 50:0
6 141:47
5 102:58
2 29:39
3 64:54
2 35:32
2 42:21
3 56:17
1 6:59
2 44:53
1 11:59
2 33:52
2 34:47
4 76:24
6 138:5
4 85:42
4 76:0
1 21:35
2 40:0
4 90:42
2 40:29
3 52:55
3 48:5
4 81:2
4 79:5
2 42:34
1 5:14
1 6:6
1 23:27
2 40:11
1 0:3
3 49:26
1 0:16
4 87:17
4 86:48
2 36:38
3 54:18
4 81:17
3 65:45
1 3:11
5 110:22
5 100:50
1 14:58
3 62:34
2 26:43
2 31:37
1 16:43
2 45:20
5 106:21
1 4:8
2 46:19
2 38:39
2 37:32
3 69:29
2 43:48
3 64:17
3 54:48
2 24:58
1 17:52
2 37:6
3 49:23
4 72:12
2 30:2
3 56:50
1 20:47
3 69:21
1 1:7
3 49:9
1 0:0
4 86:28
3 67:7
2 31:15
3 50:37
2 46:53
1 17:35
4 95:2
3 60:59
1 20:10
4 93:31
1 19:36
2 39:48
5 101:37
7 162:21
3 49:25
1 17:1
2 24:10
1 11:18
4 72:21
3 64:9
2 30:0
3 60:12
1 11:2
1 5:1
3 49

2 34:37
7 167:47
3 59:38
3 48:50
2 28:50
5 117:53
4 81:36
2 36:39
1 10:2
2 28:22
4 75:29
1 1:5
2 45:25
4 91:43
1 4:45
3 61:57
2 42:54
3 50:26
2 42:28
2 26:6
1 0:0
2 31:10
3 65:45
1 0:0
2 44:24
1 21:19
5 103:42
1 0:0
1 20:20
3 70:20
2 33:39
2 32:13
4 92:57
1 8:30
2 34:24
2 34:7
2 43:33
5 105:29
3 57:31
1 10:6
1 21:25
5 105:41
2 31:27
4 77:7
4 82:32
4 87:57
5 115:18
1 18:46
4 73:29
1 2:11
3 64:33
3 53:34
2 47:24
3 56:18
2 42:10
3 65:19
1 22:58
1 5:55
1 10:13
3 49:39
1 10:14
3 55:54
1 15:54
4 87:56
3 60:45
1 6:22
4 82:54
2 24:50
2 36:50
3 49:9
1 22:32
2 46:9
5 111:11
2 39:1
3 55:33
3 67:10
1 21:7
2 38:7
3 62:2
3 60:46
2 32:30
2 36:13
1 1:19
4 88:47
3 71:37
5 111:21
2 46:30
3 58:33
3 63:16
3 50:23
2 31:21
3 63:6
3 58:15
3 62:28
3 66:36
3 65:10
1 11:28
2 47:1
3 53:49
1 18:0
3 59:25
2 29:46
1 17:46
2 31:30
4 75:0
1 21:14
3 67:43
3 55:33
3 64:33
2 33:37
2 28:25
1 12:51
2 35:32
3 56:24
2 47:2
3 63:43
2 25:59
4 72:35
3 69:7
2 31:24
3 53:0
3 61:4
1 22:6
2 34:27
2 43:41
4 93:31
6 140:40
2 25:48
3

In [12]:
df_test_days = pd.DataFrame(test_days, columns=['DayCount']).reset_index()

In [13]:
df_test_days = df_test_days.drop(index=[0])

In [14]:
df_test_days = df_test_days.rename(columns={'index':'일수'})

In [15]:
df_test_days.to_excel('test_data_daycount.xlsx')

In [16]:
# Bug check
for idx, df in enumerate(df_train_data):
    if df[df['Parameter'] == 'isFlu'].shape[0] != 1 : display(df)

In [17]:
print(len(df_train_data), len(df_test_data), len(df_train_data+df_test_data))

5325 1332 6657


In [178]:
for idx, df_data in enumerate(df_train_data):
    digit = str(df_data['Parameter'].shape[0])
    if len(digit) == 3 and digit[0] >= '2': 
        print(df_data.shape[0])

253
244
201
226


In [179]:
def time_to_minute(time):
    hour_minute = time.split(':')
    return int(hour_minute[0]) * 60 + int(hour_minute[1])

In [190]:
def add_td_column(df):
    df_time_delta = pd.DataFrame(columns=['time_delta'])
    for param, grp in df.groupby('Parameter'):
        grp = grp.sort_values('Minute')
        grp['time_delta'] = grp['Minute'] - grp['Minute'].shift(fill_value=grp['Minute'].iloc[0])
        df_time_delta = pd.concat([df_time_delta, grp['time_delta'].to_frame()])
    
    return pd.merge(df, df_time_delta, left_index=True, right_index=True)

In [209]:
## 포멧좀 다시 만들어보자 ~
# def make_input_and_gt(df_data):
#     max_seq_len = 300
#     input_datas = np.zeros([1, 16, 2, 80])
#     gt_datas = []
    
#     for idx_chk, df_piece in tqdm(enumerate(df_data)):
#         input_data = np.zeros([1, 17])
#         df_piece['Minute'] = df_piece['Time'].apply(time_to_minute)
#         df_piece['Parameter_num'] = df_piece['Parameter'].apply(lambda x: input_dict[x])
#         df_piece = add_td_column(df_piece)
#         gt_datas.append([df_piece[df_piece['Parameter'] == 'isFlu']['Value'].iloc[0]])
#         df_piece = df_piece[df_piece['Parameter'] != 'isFlu'].sort_values(['Parameter_num', 'Minute'])
#         display(df_piece)
#         break
#         for idx, rows in df_piece.iterrows():
#             param_num = rows['Parameter_num']
#             minute = rows['Minute']
#             value = rows['Value']
#             input_data[0, param_num] = value
#             input_data[0, 16] = minute
        
#         input_data = np.array(input_data)
#         input_datas = np.concatenate((input_datas, input_data), axis=0)
# #         print(input_datas.shape)
#     return torch.Tensor(input_datas[1:].astype(np.float32)), torch.Tensor(np.array(gt_datas))

In [208]:
input_train_datas, gt_train_datas = make_input_and_gt(df_train_data)
input_test_datas, gt_test_datas = make_input_and_gt(df_test_data)

0it [00:00, ?it/s]

,Time,Parameter,Value,Minute,Parameter_num,time_delta
7,0:0,age_2to5,1.0,0,2,0
3,0:0,dew_point_avg,-13.0,0,5,0
1,0:0,humidity_avg,41.0,0,6,0
2,0:0,pressure_avg,1012.8,0,7,0
8,0:0,surveillance,6.7,0,8,0
5,0:0,temperature_min,-2.4,0,10,0
4,0:0,temperature_max,5.5,0,11,0
6,0:0,temperature_avg,0.9,0,12,0
0,0:0,reducer,3.5,0,13,0
9,6:40,reducer,3.5,400,13,400


0it [00:00, ?it/s]
0it [00:00, ?it/s]

,Time,Parameter,Value,Minute,Parameter_num,time_delta
7,0:0,age_0to2,1.0,0,1,0
3,0:0,dew_point_avg,22.2,0,5,0
1,0:0,humidity_avg,60.6,0,6,0
2,0:0,pressure_avg,998.2,0,7,0
8,0:0,surveillance,4.5,0,8,0
5,0:0,temperature_min,28.0,0,10,0
4,0:0,temperature_max,34.9,0,11,0
6,0:0,temperature_avg,31.1,0,12,0
0,0:0,reducer,5.0,0,13,0
11,8:24,reducer,5.0,504,13,504


0it [00:00, ?it/s]


In [39]:
input_train_datas.shape, gt_train_datas.shape

(torch.Size([5325, 17]), torch.Size([5325, 1]))

In [99]:
input_train_surveilances = input_train_datas[:, 8]
input_test_surveilances = input_test_datas[:, 8]

In [100]:
input_train_surveilances.shape

torch.Size([5325])

In [101]:
### time delta 제거해보기
input_train_datas_without_td = input_train_datas[:, :16]
input_test_datas_without_td = input_test_datas[:, :16]

In [102]:
input_train_datas_without_td.shape, input_test_datas_without_td.shape

(torch.Size([5325, 16]), torch.Size([1332, 16]))

In [103]:
class FluDataset(Dataset):
    def __init__(self, input_data, gt, transform=None, target_transform=None):
        self.input_data = input_data
        self.gt = gt
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, idx):
        return self.input_data[idx], self.gt[idx]

In [104]:
training_data = FluDataset(input_train_surveilances, gt_train_datas)
test_data = FluDataset(input_test_surveilances, gt_test_datas)

train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True) 
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=True)

In [117]:
class FluBaselineModel(nn.Module):
    def __init__(self):
        super(FluBaselineModel, self).__init__()
        self.fcs = nn.Sequential(
#             nn.BatchNorm1d(1),
            nn.Linear(1, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Linear(20, 40),
            nn.BatchNorm1d(40),
            nn.ReLU(),
            nn.Linear(40, 80),
            nn.BatchNorm1d(80),
            nn.ReLU(),
            nn.Linear(80, 1),
            nn.Sigmoid()
        )
    
    def forward(self, pghd):
        pghd = pghd.view(-1, 1)
#         print(pghd.shape)
        pred = self.fcs(pghd)
        return pred

In [118]:
model = FluBaselineModel()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [119]:
len(train_dataloader), len(test_dataloader)

(666, 1332)

In [120]:
train_running_loss = 0.0
test_running_loss = 0.0
num_epochs = 3000

model.train()
for epoch in tqdm(range(num_epochs)):
    print()
    for idx, data in enumerate(train_dataloader, 0):
#     for idx, (train_data, test_data) in enumerate(zip(train_dataloader, test_dataloader)):
        train_inputs, train_labels = data
#         test_inputs, test_labels = test_data
#         print(train_inputs.shape)
        
        optimizer.zero_grad()
        
        train_outputs = model(train_inputs)
        train_loss = criterion(train_outputs, train_labels)
        train_loss.backward()
        optimizer.step()
        
#         test_outputs = model(test_inputs)
#         test_loss = criterion(test_outputs, test_labels)
        
        train_running_loss += train_loss.item()
#         test_running_loss += test_loss.item()
        
        if idx % 100 == 99:
            print(f'Train] epoch] {epoch+1}, idx] {idx} loss] {train_running_loss}')
#             print(f'Test] epoch] {epoch+1}, idx] {idx} loss] {test_running_loss}')
            print()
            train_running_loss = 0.0
            test_running_loss = 0.0
print('Finished Training')

  0%|          | 0/3000 [00:00<?, ?it/s]


Train] epoch] 1, idx] 99 loss] 58.23650813102722

Train] epoch] 1, idx] 199 loss] 46.619090273976326

Train] epoch] 1, idx] 299 loss] 44.1877331584692

Train] epoch] 1, idx] 399 loss] 49.28875160217285

Train] epoch] 1, idx] 499 loss] 49.10065324604511



  0%|          | 1/3000 [00:00<46:17,  1.08it/s]

Train] epoch] 1, idx] 599 loss] 46.358672961592674


Train] epoch] 2, idx] 99 loss] 75.71785144507885

Train] epoch] 2, idx] 199 loss] 47.32261082530022

Train] epoch] 2, idx] 299 loss] 42.73398381471634

Train] epoch] 2, idx] 399 loss] 46.19388861954212

Train] epoch] 2, idx] 499 loss] 50.14299134910107

Train] epoch] 2, idx] 599 loss] 43.50314071774483



  0%|          | 2/3000 [00:01<48:04,  1.04it/s]

...

 10%|▉         | 285/3000 [04:23<42:13,  1.07it/s]

Train] epoch] 285, idx] 599 loss] 38.81166846677661


Train] epoch] 286, idx] 99 loss] 54.47009505331516

Train] epoch] 286, idx] 199 loss] 35.946847319602966



 10%|▉         | 285/3000 [04:23<41:53,  1.08it/s]

Train] epoch] 286, idx] 299 loss] 36.89218036457896

Train] epoch] 286, idx] 399 loss] 34.00505978614092



KeyboardInterrupt: 

In [121]:
model.eval()
preds = []
labels = []
for idx, data in enumerate(test_dataloader, 0):
    test_inputs, test_labels = data
    test_outputs = model(test_inputs)
    labels.append(test_labels.item())
    preds.append(test_outputs.item())
preds = np.asarray(preds)
labels = np.asarray(labels)

In [122]:
labels[:10], preds[:10]

(array([1., 0., 0., 0., 0., 0., 1., 0., 0., 1.]),
 array([0.47305849, 0.15169111, 0.10831822, 0.13967714, 0.12847234,
        0.16865295, 0.34827045, 0.10831822, 0.1360736 , 0.22022463]))

In [123]:
auc_score = roc_auc_score(labels, preds)

In [124]:
acc = accuracy_score(labels, (preds > 0.5).astype(int))

In [125]:
acc, auc_score

(0.801051051051051, 0.9182129815932634)